In [1]:
# import numpy 
import numpy as np

In [13]:
# Creating the Tic Tac Toes Game

class TicTacToe:
    def __init__(self):
        self.row_count=3
        self.column_count=3
        self.action_size=self.row_count*self.column_count
        
    def get_init_state(self):
        return np.zeros((self.row_count,self.column_count))
    def get_next_state(self,state,action,player):
        row = action // self.column_count
        column = action % self.column_count
        state[row,column]=player
        return state
    def get_valid_moves(self,state):
        return(state.reshape(-1) == 0).astype(np.uint8)
    def check_win(self,state,action):
        row = action // self.column_count
        column = action % self.column_count
        player = state[row,column]
        
        return(
            np.sum(state[row,:]) == player *(self.column_count)
            or np.sum(state[:,column]) == player * (self.row_count)
            or np.sum(np.diag(state))==player * self.row_count
            or np.sum(np.diag(np.flip(state,axis=0 ))) == player * self.row_count
        )
    
    def get_val_and_end(self,state,action):
        if self.check_win(state,action):
            return 1,True
        if np.sum(self.get_valid_moves(state))==0:
            return 0, True
        return 0, False
    def get_opp(self,player):
        return -player

In [15]:
#Simulating tic tac toe playing to ensure game works correctly
ticTac= TicTacToe()
player=1
state = ticTac.get_init_state()

while True:
    print(state)
    valid_moves= ticTac.get_valid_moves(state)
    print("valid moves", [i for i in range(ticTac.action_size) if valid_moves[i]==1])
    
    action = int(input(f"{player:}"))
    
    if valid_moves[action]==0:
        print("action not valid")
        continue
    state = ticTac.get_next_state(state,action,player)
    
    value,is_terminal = ticTac.get_val_and_end(state,action)
    
    if is_terminal:
        print(state)
        if value==1:
            print(player,"won")
        else:
            print("tie")
        break;
    player=ticTac.get_opp(player)
    

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
valid moves [0, 1, 2, 3, 4, 5, 6, 7, 8]
15
[[0. 0. 0.]
 [0. 0. 1.]
 [0. 0. 0.]]
valid moves [0, 1, 2, 3, 4, 6, 7, 8]
-10
[[-1.  0.  0.]
 [ 0.  0.  1.]
 [ 0.  0.  0.]]
valid moves [1, 2, 3, 4, 6, 7, 8]
18
[[-1.  0.  0.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]]
valid moves [1, 2, 3, 4, 6, 7]
-13
[[-1.  0.  0.]
 [-1.  0.  1.]
 [ 0.  0.  1.]]
valid moves [1, 2, 4, 6, 7]
12
[[-1.  0.  1.]
 [-1.  0.  1.]
 [ 0.  0.  1.]]
1 won


In [ ]:
# Monte Carlo Tree Search

class Node:
    def __init__(self,game,args,state, parent=None,action_take=None):
        self.game=game
        self.args=args
        self.state=state
        self.parent=parent
        self.action_taken=action_taken
        
        self.children=[]
        self.expandable_moves=game.get_valid_moves(state)
        
        self.visit_count=0
        self.value_sum=0
    def is_fully_expanded(self):
        return np.sum(self.expandable_moves)==0 and len(self.children)>0
    
    def select(self):
        best_child=None
        best_ucb=-np.inf
        
        for child in self.children:
            ucb = self.get_ucb(child)
            if(ucb>best_ucb):
                best_child=child
                best_ucb=ucb
        return best_child
    
    def get_ucb(self,child):
        
        
        
        
class MCTS:
    def __init__(self,game,args):
        self.game=game
        self.args=args
        
    def search(self,state):
        #define root
        root=Node(self,game,self.args,state)
        #Search iterations
        for search in range(self.args['num_searches']):
            node = root
            while node.is_fully_expanded():
                node=node.select()
            